In [1]:
# data Analysis
import pandas as pd
import numpy as np
import random as rng

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inlinewhich pip


#SciKit Learn Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

UsageError: unrecognized arguments: pip


In [2]:
from subprocess import check_output
print(check_output(["ls", "Scrapers/data"]).decode("utf8"))
data = pd.read_csv("Scrapers/data/finalout.csv")

fighter_profile_template.json
fight_json
finalout.csv
json_with_other_template
move_files.py
profile_json
profile_links
results_record.json
time_bound_profiles



In [6]:
data.describe()

,BPrev,BStreak,B_ID,B__Round1_Grappling_Reversals_Landed,B__Round1_Grappling_Standups_Landed,B__Round1_Grappling_Submissions_Attempts,B__Round1_Grappling_Takedowns_Attempts,B__Round1_Grappling_Takedowns_Landed,B__Round1_Strikes_Body Significant Strikes_Attempts,B__Round1_Strikes_Body Significant Strikes_Landed,...,R__Round5_TIP_Distance Time,R__Round5_TIP_Ground Control Time,R__Round5_TIP_Ground Time,R__Round5_TIP_Guard Control Time,R__Round5_TIP_Half Guard Control Time,R__Round5_TIP_Misc. Ground Control Time,R__Round5_TIP_Mount Control Time,R__Round5_TIP_Neutral Time,R__Round5_TIP_Side Control Time,R__Round5_TIP_Standing Time
count,2169.000000,2169.000000,2169.000000,1529.000000,1529.000000,1529.000000,1529.000000,1529.000000,1529.000000,1529.000000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,2.263255,0.744583,2124.675888,0.073250,1.064094,0.547417,3.618051,1.548071,14.606279,9.671027,...,267.532000,35.528000,68.256000,6.756000,4.240000,4.644000,12.916000,279.712000,4.688000,321.988000
std,2.428562,1.148659,705.206120,0.300301,1.492503,1.015203,4.989055,2.303639,15.660815,10.597081,...,194.949713,77.862929,102.307192,25.276473,13.490678,13.255493,40.568215,196.521919,21.726824,220.780461
min,0.000000,0.000000,129.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1905.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.000000,...,160.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,163.000000,0.000000,194.000000
50%,2.000000,0.000000,2229.000000,0.000000,1.000000,0.000000,2.000000,1.000000,9.000000,6.000000,...,232.000000,2.000000,17.000000,0.000000,0.000000,0.000000,0.000000,267.000000,0.000000,298.000000
75%,3.000000,1.000000,2733.000000,0.000000,2.000000,1.000000,5.000000,2.000000,20.000000,13.000000,...,299.000000,36.250000,109.000000,0.000000,0.000000,1.000000,3.000000,301.000000,0.000000,370.750000
max,13.000000,10.000000,3196.000000,3.000000,13.000000,7.000000,47.000000,20.000000,124.000000,72.000000,...,1259.000000,633.000000,666.000000,144.000000,91.000000,62.000000,273.000000,1291.000000,200.000000,1473.000000


In [7]:
data.describe(include=['O'])

,B_Name,Date,R_Name,winby,winner
count,2169,2169,2169,2143,2169
unique,918,352,814,3,4
top,Kevin Lee,10/08/2018,Donald Cerrone,DEC,red
freq,10,75,14,1053,1240


### Some Notes to observe
- Red Side seems to win slightly more than blue (1240/2169 = 57.16%)
- Donald Cerrone fights on Red side more than any other fighter, with 14 fights
- Kevin Lee fights on Blue side more than any other fighter with 10
- Most Fights are won by decision

In [9]:
list(data)

['BPrev',
 'BStreak',
 'B_ID',
 'B_Name',
 'B__Round1_Grappling_Reversals_Landed',
 'B__Round1_Grappling_Standups_Landed',
 'B__Round1_Grappling_Submissions_Attempts',
 'B__Round1_Grappling_Takedowns_Attempts',
 'B__Round1_Grappling_Takedowns_Landed',
 'B__Round1_Strikes_Body Significant Strikes_Attempts',
 'B__Round1_Strikes_Body Significant Strikes_Landed',
 'B__Round1_Strikes_Body Total Strikes_Attempts',
 'B__Round1_Strikes_Body Total Strikes_Landed',
 'B__Round1_Strikes_Clinch Body Strikes_Attempts',
 'B__Round1_Strikes_Clinch Body Strikes_Landed',
 'B__Round1_Strikes_Clinch Head Strikes_Attempts',
 'B__Round1_Strikes_Clinch Head Strikes_Landed',
 'B__Round1_Strikes_Clinch Leg Strikes_Attempts',
 'B__Round1_Strikes_Clinch Leg Strikes_Landed',
 'B__Round1_Strikes_Clinch Significant Kicks_Attempts',
 'B__Round1_Strikes_Clinch Significant Kicks_Landed',
 'B__Round1_Strikes_Clinch Significant Punches_Attempts',
 'B__Round1_Strikes_Clinch Significant Punches_Landed',
 'B__Round1_Strike